<a href="https://colab.research.google.com/github/Kamaljit12/HuggingFace/blob/main/Emothion_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pwd

'/content'

In [ ]:
ls "/content/drive/MyDrive/Datasets/top_10000.csv"

/content/drive/MyDrive/Datasets/top_10000.csv


In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split

## load the csv file
df = pd.read_csv("/content/drive/MyDrive/Datasets/top_10000.csv")

In [ ]:
df.head()

,cleaned_senetences,emotion
0,i just feel really helpless and heavy hearted,fear
1,ive enjoyed being able to slouch about relax a...,sad
2,i gave up my internship with the dmrg and am f...,fear
3,i dont know i feel so lost,sad
4,i am a kindergarten teacher and i am thoroughl...,fear


In [ ]:
df.rename(columns={"cleaned_senetences": "sentence"}, inplace=True)

In [ ]:
# shape of the data
df.shape

(9999, 2)

In [ ]:
# info of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9999 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  9999 non-null   object
 1   emotion   9999 non-null   object
dtypes: object(2)
memory usage: 492.4+ KB


In [ ]:
# check duplicates
df.duplicated().sum()

0

In [ ]:
# drop duplicated
df.drop_duplicates(inplace=True)

In [ ]:
# check duplicates
df.duplicated().sum()

0

In [84]:
# ## clean the text: Remvoe special characters and lowercsting

# def clean_text(text):
#     # lower case of the text
#     text = text.lower()
#     # regex to remove special characters
#     text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
#     return text


In [ ]:
# # applied the above cleaning fucntion to clean data
# df["sentence"] = df['cleaned_senetence'].apply(clean_text)

In [ ]:
df.head()

,sentence,emotion
0,i just feel really helpless and heavy hearted,fear
1,ive enjoyed being able to slouch about relax a...,sad
2,i gave up my internship with the dmrg and am f...,fear
3,i dont know i feel so lost,sad
4,i am a kindergarten teacher and i am thoroughl...,fear


In [ ]:
# function to count word of the sentences
def text_counts(text):
  words = text.split(" ")
  word_count = len(words)
  return word_count

In [ ]:
df["total_words"] = df["sentence"].apply(text_counts)

In [ ]:
## maximum number of the sentences
df['total_words'].max()

65

In [ ]:
## counts of emotion category
df["emotion"].value_counts()

,count
emotion,
joy,3395
sad,2935
anger,1367
fear,1149
love,789
suprise,364


In [ ]:
# split the data for the training and for testing
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
#!pip install datasets

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

In [ ]:
## initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Tokenize the input prompts and generated recipes
train_encodings = tokenizer(list(train_data['sentence']), truncation=True, padding=True, max_length=67)
train_labels = tokenizer(list(train_data['emotion']), truncation=True, padding=True, max_length=67)

test_encodings = tokenizer(list(test_data['sentence']), truncation=True, padding=True, max_length=67)
test_labels = tokenizer(list(test_data['emotion']), truncation=True, padding=True, max_length=67)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:

# Create a custom dataset for use in the Trainer
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels['input_ids']
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_labels['input_ids']
})


In [76]:
# Set up training arguments with validation logging
training_args = TrainingArguments(
    output_dir='./results',  # Output directory
    num_train_epochs=5,  # Number of training epochs
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,  # Batch size for evaluation
    warmup_steps=500,  # Warmup steps
    weight_decay=0.01,  # Weight decay
    logging_dir='./logs',  # Logging directory
    logging_steps=10,
    evaluation_strategy="steps",  # Evaluate during training
    eval_steps=500,  # Evaluate every 500 steps
    save_steps=1000,  # Save model checkpoints every 1000 steps
    load_best_model_at_end=True,  # Load the best model when finished
)

# Initialize Trainer
trainer = Trainer(
    model=model,  # The model to train
    args=training_args,  # Training arguments
    train_dataset=train_dataset,  # Training dataset
    eval_dataset=test_dataset,  # Evaluation dataset
)

# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
500,0.381700,0.254902
1000,0.215000,0.166012
1500,0.149100,0.131039
2000,0.114000,0.108180
2500,0.150000,0.094888
3000,0.157200,0.084361
3500,0.096400,0.083053
4000,0.110300,0.076312


Step,Training Loss,Validation Loss
500,0.381700,0.254902
1000,0.215000,0.166012
1500,0.149100,0.131039
2000,0.114000,0.108180
2500,0.150000,0.094888
3000,0.157200,0.084361
3500,0.096400,0.083053
4000,0.110300,0.076312
4500,0.092200,0.074935
5000,0.082300,0.075394


Could not locate the best model at ./results/checkpoint-4500/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=5000, training_loss=0.5699428015232086, metrics={'train_runtime': 558.976, 'train_samples_per_second': 71.55, 'train_steps_per_second': 8.945, 'total_flos': 708341188362240.0, 'train_loss': 0.5699428015232086, 'epoch': 5.0})

In [78]:

# Save the fine-tuned model
model.save_pretrained('/content/drive/MyDrive/Datasets/fine_tuned_t5_emotion_classifier')
tokenizer.save_pretrained('/content/drive/MyDrive/Datasets/fine_tuned_t5_emotion_classifier')

('/content/drive/MyDrive/Datasets/fine_tuned_t5_emotion_classifier/tokenizer_config.json',
 '/content/drive/MyDrive/Datasets/fine_tuned_t5_emotion_classifier/special_tokens_map.json',
 '/content/drive/MyDrive/Datasets/fine_tuned_t5_emotion_classifier/spiece.model',
 '/content/drive/MyDrive/Datasets/fine_tuned_t5_emotion_classifier/added_tokens.json')

## Emotion Classifier

In [80]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

## laod the model and tokernizer

mdoel = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Datasets/fine_tuned_t5_emotion_classifier")
tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/Datasets/fine_tuned_t5_emotion_classifier")

In [81]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [82]:
device

device(type='cuda')

In [ ]:
# model.to(device)

In [85]:

def clean_text(text):
    # lower case of the text
    text = text.lower()
    # regex to remove special characters
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    return text

In [87]:
# Function to generate a recipe from a prompt
def generate_emotion(input, model, tokenizer, max_length):
    prompt = clean_text(input)  # Clean the input prompt
    inputs = tokenizer(input, return_tensors="pt", padding=True, truncation=True, max_length=67)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Generate the recipe
    output = model.generate(input_ids, attention_mask=attention_mask, max_length=max_length, num_return_sequences=1)

    return tokenizer.decode(output[0], skip_special_tokens=True)





In [88]:
# Test the recipe generation
sentence = "i feel about the monsters who did that to us i know that fanatics no matter how hateful amp murderous don t make an army"
result = generate_emotion(input=sentence, model=model, tokenizer=tokenizer, max_length=67)
print(result)


anger


In [89]:
import pandas as pd

In [92]:
# df = pd.read_csv("/content/drive/MyDrive/Datasets/top_10000.csv")